In [17]:
import os
import time
import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv

import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

load_dotenv()

True

In [18]:
import logging
from logging.handlers import RotatingFileHandler

logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

file_formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(levelname)s - %(message)s")
log_dir = os.path.join(os.getcwd(), ".." , "logs")
file_handler = RotatingFileHandler(
    filename=os.path.join(log_dir, "scraper.log"),
    maxBytes=10*1024*1024,
    backupCount=5
)
file_handler.setLevel(logging.DEBUG)
file_handler.setFormatter(file_formatter)

logger.addHandler(file_handler)

In [19]:
import traceback


def check_for_new_offers(location, url):
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        search_result = soup.find("h2", class_="SearchResults-desktop")

        if search_result:
            message = search_result.get_text(strip=True)

            if "Aucun logement" not in message:
                individual_appartments_counter = get_inidividual_appartment_offers_count(
                    soup)

                if individual_appartments_counter > 0:
                    log_content(location, message)
                    send_email(location, message, url, os.getenv("RECEIVER_EMAIL"))
                    send_email(location, message, url, "simad3647@gmail.com")


def get_inidividual_appartment_offers_count(soup):
    card_classes = "fr-col-12 fr-col-sm-6 fr-col-md-4 svelte-11sc5my fr-col-lg-4"
    aprtment_cards = soup.find_all("li", class_=card_classes)

    individual_appartments_counter = 0
    for card in aprtment_cards:
        card_detail_classes = "fr-card__detail fr-icon-group-fill"
        card_detail = card.find("p", class_=card_detail_classes)
        card_colocation_or_individual_text = card_detail.get_text(strip=True)

        if card_colocation_or_individual_text == "Colocation":
            continue
        elif card_colocation_or_individual_text == "Individuel":
            apartment_title = card.find(
                "h3", class_="fr-card__title").get_text(strip=True)
            if check_if_new_offers_available(card):
                log_content("Individual Appartment found", apartment_title)
                individual_appartments_counter += 1

    return individual_appartments_counter


def check_if_new_offers_available(card):
    apartment_link = card.find("a")["href"]
    appartment_offer_page = f"""https://trouverunlogement.lescrous.fr{
        apartment_link}"""

    response = requests.get(appartment_offer_page)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")

        button_container = soup.find("div", class_="fr-mb-3w")
        button = button_container.find(
            "button", class_="svelte-eq6rxe fr-btn")
        span = button.find("span", class_="svelte-eq6rxe")
        span_text = span.get_text(strip=True)
        if span_text != "Indisponible":
            return True
        
    return False


def log_content(location, message):
    log_entry = f"{location.ljust(60)}: {message}\n"
    logger.info(log_entry)


def send_email(location, message, url, receiver_email):
    sender_email = os.getenv("SENDER_EMAIL")
    sender_password = os.getenv("SENDER_PASSWORD")

    subject = "New Apartment Offers"
    body = f"There are new apartment offers in \
        {location}: {message}\nurl : {url}"

    msg = MIMEMultipart()
    msg["From"] = sender_email
    msg["To"] = receiver_email
    msg["Subject"] = subject

    msg.attach(MIMEText(body, "plain"))

    try:
        with smtplib.SMTP("smtp.gmail.com", 587) as server:
            server.starttls()
            server.login(sender_email, sender_password)
            text = msg.as_string()
            server.sendmail(sender_email, receiver_email, text)
            print(f"Email notification sent for {location}")
    except Exception as e:
        print(f"Error sending email: {e}")
        traceback.print_exc()

In [20]:
url_1 = "https://trouverunlogement.lescrous.fr/tools/36/search?bounds=2.4130316_48.6485333_2.4705092_48.6109217"

locations = {
    "Evry": url_1,
}

In [21]:
logger.info("Starting the scraper")
while True:
    for location, url in locations.items():
        check_for_new_offers(location, url)

    time.sleep(60)

KeyboardInterrupt: 